In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [35]:
# 隨機種子確保可重現性
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [36]:
# 設置環境變量確保使用單一CPU核心
import os
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# 清除Keras的後端會話
tf.keras.backend.clear_session()

In [37]:
# 載入資料
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 訓練和驗證資料
X = train.iloc[:, 1:].values.reshape(-1, 28, 28, 1)
y = to_categorical(train['label'].values)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=seed)

# 測試資料
X_test = test.values.reshape(-1, 28, 28, 1)

In [38]:
# 建立模型
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [40]:
# 訓練模型
model = create_model()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history = model.fit(X_train, y_train, epochs=200, batch_size=200, validation_data=(X_val, y_val),
                    callbacks=[early_stopping, reduce_lr], verbose=2)

Epoch 1/200
189/189 - 5s - loss: 2.4756 - accuracy: 0.4958 - val_loss: 0.5427 - val_accuracy: 0.9295 - lr: 0.0010 - 5s/epoch - 25ms/step
Epoch 2/200
189/189 - 1s - loss: 0.8031 - accuracy: 0.8266 - val_loss: 0.3875 - val_accuracy: 0.9648 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 3/200
189/189 - 1s - loss: 0.5802 - accuracy: 0.9002 - val_loss: 0.3429 - val_accuracy: 0.9705 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 4/200
189/189 - 1s - loss: 0.4835 - accuracy: 0.9276 - val_loss: 0.3089 - val_accuracy: 0.9779 - lr: 0.0010 - 1s/epoch - 7ms/step
Epoch 5/200
189/189 - 1s - loss: 0.4226 - accuracy: 0.9421 - val_loss: 0.2808 - val_accuracy: 0.9814 - lr: 0.0010 - 1s/epoch - 8ms/step
Epoch 6/200
189/189 - 1s - loss: 0.3818 - accuracy: 0.9483 - val_loss: 0.2641 - val_accuracy: 0.9793 - lr: 0.0010 - 1s/epoch - 7ms/step
Epoch 7/200
189/189 - 1s - loss: 0.3473 - accuracy: 0.9550 - val_loss: 0.2431 - val_accuracy: 0.9848 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 8/200
189/189 - 1s - loss: 0.3229 - accur

In [41]:
# 進行預測
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# 輸出檔案
submission = pd.DataFrame({'ImageId': np.arange(1, len(predicted_labels) + 1), 'Label': predicted_labels})
submission.to_csv('submission.csv', index=False)

print("Submission file has been created!")

875/875 [==============================] - 2s 2ms/step
Submission file has been created!
